# Answer to Question 4 of Exercises

In [1]:
# import pandas and numpy in case we need them
import pandas as pd
import numpy as np

In [2]:
from xml.etree import ElementTree as ET

In [3]:
# find the unique tag names that are in the xml file

document_tree = ET.parse( './data/mondial_database.xml' )
root = document_tree.getroot()
tags = []
for elementtag in document_tree.getiterator():
    tagname = elementtag.tag
    if (tagname in tags[:]):
        tags = tags  
    else: 
        tags.append(tagname)

In [4]:
tags

['mondial',
 'country',
 'name',
 'population',
 'population_growth',
 'infant_mortality',
 'gdp_total',
 'gdp_agri',
 'gdp_ind',
 'gdp_serv',
 'inflation',
 'unemployment',
 'indep_date',
 'government',
 'encompassed',
 'ethnicgroup',
 'religion',
 'language',
 'border',
 'city',
 'latitude',
 'longitude',
 'located_at',
 'localname',
 'province',
 'area',
 'located_on',
 'dependent',
 'elevation',
 'continent',
 'organization',
 'abbrev',
 'established',
 'members',
 'sea',
 'located',
 'depth',
 'river',
 'to',
 'length',
 'source',
 'estuary',
 'through',
 'from',
 'mountains',
 'lake',
 'island',
 'islands',
 'mountain',
 'desert',
 'airport',
 'gmtOffset']

In [5]:
# Take note of the tag names: lake, depth, river, length, airport, elevation
# Pull up the mondial xml file and do an edit/search on these tags to obtain the structure of the tree
# Note that the geographical data is not under the element 'country'
# Note that the country will need to be extracted from the attribute 'country=' of tag 'located' which will return a country code
# The name of the country will then be found by searching on tag 'country' with attribute car_code= 
# Note that rivers and lakes may be in more than one country

In [6]:
# create a lookup table for country code/name
lookuptable = []
for country in root.iterfind('country'):
    country_name = country.find('name').text
    country_code = country.get('car_code')
    lookuptable.append([country_code, country_name])
df_countrytable = pd.DataFrame(lookuptable, columns= ['code','country'])

In [7]:
# set the code column as the index
df_countrytable_indexed = df_countrytable.set_index('code').sort_index()
df_countrytable_indexed.head()

country
code                     
A                 Austria
AFG           Afghanistan
AG    Antigua and Barbuda
AL                Albania
AMSA       American Samoa

# Find airport at highest elevation

In [10]:

# create a list of lists containing the aiport name, elevation, and countrycode
airports = []
for airport in root.iterfind('airport'):
    airportname = airport.find('name').text
    elevation = airport.find('elevation').text   
    country_code = airport.get('country')
    if elevation != None:
        airports.append([elevation, airportname, country_code])
df_airports = pd.DataFrame(airports, columns= ['elevation','airport','countrycode'])  

In [11]:
df_airports.head(10)

elevation               airport countrycode
0       977                 Herat         AFG
1      1792            Kabul Intl         AFG
2        38          Tirana Rinas          AL
3       811  Cheikh Larbi Tebessi          DZ
4       822         Batna Airport          DZ
5         6               Soummam          DZ
6      1377           Tamanrasset          DZ
7        88                Biskra          DZ
8       691  Mohamed Boudiaf Intl          DZ
9      1024     Ain Arnat Airport          DZ

In [12]:
# set the datatype of column 'elevation' to integer type
df_airports['elevation'] = df_airports['elevation'].astype('int')

In [13]:
# sort the dataframe by 'elevation' in descending order
df_airports_sorted = df_airports.sort_values(by='elevation', ascending=False)

In [14]:
# define series s as the top row of the airports data frame
s = df_airports_sorted.iloc[0]

In [15]:
s

elevation              4063
airport        El Alto Intl
countrycode             BOL
Name: 80, dtype: object

In [16]:
# use the countrycode from s to locate the country name in the data frame table df_countrytable_indexed
df_countrytable_indexed.loc[s.loc['countrycode']] 

country    Bolivia
Name: BOL, dtype: object

In [17]:
# add 'country' to s so s will contain elevation, airport name, country code, and country name
s['country'] = df_countrytable_indexed.loc[s.loc['countrycode']] 

C:\Users\sgior\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\sgior\Anaconda2\lib\site-packages\pandas\core\series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [18]:
s

elevation                                             4063
airport                                       El Alto Intl
countrycode                                            BOL
country        country    Bolivia
Name: BOL, dtype: object
Name: 80, dtype: object

# Find Deepest Lake

In [8]:
lakes = []
for lake in root.iterfind('lake'):
    lakename = lake.find('name').text
    lakedepth = lake.find('depth')   
    country_code = lake.get('country')
    if lakedepth != None:
        lakes.append([float(lakedepth.text), lakename, country_code])
df_lakes = pd.DataFrame(lakes, columns= ['depth','lake','countrycode'])    

In [9]:
df_lakes_sorted = df_lakes.sort_values(by='depth', ascending=False)

In [10]:
s = df_lakes_sorted.iloc[0]

In [14]:
index = s.loc['countrycode']

In [15]:
df_countrytable_indexed.loc[index,'country'] 

'Russia'

In [16]:
s['country'] = df_countrytable_indexed.loc[index,'country'] 

C:\Users\sgior\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
s

depth                  1637
lake           Ozero Baikal
countrycode               R
country              Russia
Name: 42, dtype: object

# Longest River

In [18]:
# create a list of lists containing river name, river length, and country code and load into a data frame called df_rivers
rivers = []
for river in root.iterfind('river'):
    rivername = river.find('name').text
    riverlength = river.find('length')   
    country_code = river.get('country')
    if riverlength != None:
        rivers.append([float(riverlength.text), rivername, country_code])
df_rivers = pd.DataFrame(rivers, columns= ['length','river','countrycode'])    

In [19]:
# sort the data frame by length of rivers in descending order
df_rivers_sorted = df_rivers.sort_values(by= 'length', ascending = False)

In [20]:
# slice off the top row which contains the longest river
s = df_rivers_sorted.iloc[0]

In [21]:
s

length             6448
river          Amazonas
countrycode    CO BR PE
Name: 174, dtype: object

In [22]:
# the countrycode contains more than one country so create a list of the codes using split on white space
list_of_codes = s['countrycode'].split()

In [23]:
# create a list of country names using the country codes as index to the data frame containing the country names
countries = []
for code in list_of_codes:
    countrycode = code
    countryname = df_countrytable_indexed.loc[code,'country'] 
    countries.append(countryname)

In [24]:
# add the list of countries to s
s['countries'] = countries

C:\Users\sgior\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [25]:
s

length                             6448
river                          Amazonas
countrycode                    CO BR PE
countries      [Colombia, Brazil, Peru]
Name: 174, dtype: object